# Importar Librerias

In [1]:
import pandas as pd
from pathlib import Path
import sqlite3

# Lectura Datos

In [2]:
DB_DIR  = Path("../data/raw")   
DB_DIR.mkdir(parents=True, exist_ok=True)
DB_PATH = DB_DIR / "online_retail.xlsx"
print(DB_PATH)
df = pd.read_excel(
    DB_PATH,
    engine="openpyxl",
    dtype={"InvoiceNo":"string","StockCode":"string"},
    parse_dates=["InvoiceDate"]
)



..\data\raw\online_retail.xlsx


# Ver primeras 5 filas

In [3]:
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Información del Data set

In [4]:

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  string        
 1   StockCode    541909 non-null  string        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2), string(2)
memory usage: 33.1+ MB
None


# Eliminar nulos en CustomerID y Description

In [5]:
df_clean = df.dropna(subset=["CustomerID"]).copy()

# Quitar facturas canceladas

In [6]:
df_clean = df_clean[~df_clean["InvoiceNo"].astype(str).str.startswith("C")]

# Filtrar cantidades y precios positivos

In [7]:
df_clean = df_clean[(df_clean["Quantity"] > 0) & (df_clean["UnitPrice"] > 0)]

# Normalizar descripción (mayúsculas)

In [8]:
df_clean["Description"] = df_clean["Description"].str.strip().str.upper()


# Información data set limpio

In [9]:
df_clean.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
df_clean = df_clean.reset_index(drop=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  string        
 1   StockCode    397884 non-null  string        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  float64       
 7   Country      397884 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2), string(2)
memory usage: 24.3+ MB


In [11]:
len(df_clean)

397884

# Path de carga para la base SQlite

In [12]:
DB_DIR  = Path("../db")   
DB_DIR.mkdir(parents=True, exist_ok=True)
DB_PATH = DB_DIR / "online_retail_clean.db"
print(DB_PATH)

..\db\online_retail_clean.db


# Cargar la base a sqlite

In [13]:
with sqlite3.connect(DB_PATH) as conn:
    df_clean.to_sql("transactions", conn, if_exists="replace", index=False, chunksize=100_000)

    #  Índices útiles para acelerar consultas
    conn.execute("CREATE INDEX IF NOT EXISTS ix_transactions_invoice ON transactions (InvoiceNo)")
    conn.execute("CREATE INDEX IF NOT EXISTS ix_transactions_customer ON transactions (CustomerID)")
    conn.execute("CREATE INDEX IF NOT EXISTS ix_transactions_date ON transactions (InvoiceDate)")
    conn.commit()

print("✅ Base creada en:", DB_PATH.resolve())


with sqlite3.connect(DB_PATH) as conn:
    n = pd.read_sql_query("SELECT COUNT(*) AS n FROM transactions", conn)
    print("Filas en 'transactions':", int(n.loc[0, "n"]))

✅ Base creada en: C:\Users\jairo\Documents\Maestria\primer semestre\Fundamentos_en_Ciencia_de_datos\Tareas\1\online-retail-project\db\online_retail_clean.db
Filas en 'transactions': 397884


In [14]:
with sqlite3.connect(DB_PATH) as conn:
    df_sql = pd.read_sql_query("SELECT * FROM transactions LIMIT 5", conn)
df_sql.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Pruebas con facturas

In [50]:
import pandas as pd

processed_df = (
    df_clean
    .loc[:, ["Country", "InvoiceDate"]]
    .assign(InvoiceDate=lambda d: pd.to_datetime(d["InvoiceDate"]))
    .rename(columns=lambda c: c.strip().lower().replace(" ", "_"))
)
processed_df


,country,invoicedate
0,United Kingdom,2010-12-01 08:26:00
1,United Kingdom,2010-12-01 08:26:00
2,United Kingdom,2010-12-01 08:26:00
3,United Kingdom,2010-12-01 08:26:00
4,United Kingdom,2010-12-01 08:26:00
...,...,...
397879,France,2011-12-09 12:50:00
397880,France,2011-12-09 12:50:00
397881,France,2011-12-09 12:50:00
397882,France,2011-12-09 12:50:00


In [51]:
out = (df_clean
       .assign(InvoiceDate=pd.to_datetime(df_clean["InvoiceDate"]),
               Revenue=lambda d: d["Quantity"] * d["UnitPrice"])
       .groupby([df_clean["Country"], df_clean["InvoiceDate"].dt.to_period("M")])["Revenue"]
       .sum()
       .rename("monthly_revenue")
       .reset_index()
       .assign(InvoiceDate=lambda d: d["InvoiceDate"].dt.to_timestamp()))

out


,Country,InvoiceDate,monthly_revenue
0,Australia,2010-12-01,1032.85
1,Australia,2011-01-01,9017.71
2,Australia,2011-02-01,14695.42
3,Australia,2011-03-01,17223.99
4,Australia,2011-04-01,771.60
...,...,...,...
282,Unspecified,2011-04-01,299.10
283,Unspecified,2011-05-01,852.68
284,Unspecified,2011-06-01,185.78
285,Unspecified,2011-07-01,798.48
